# Import Packages

In [1]:
import tabula
from re import sub
import pandas as pd
import numpy as np
import os

In [2]:
import extract as ex

# Import Data

In [3]:
pdf_path=f"{os.getcwd()}/data/discover_10K_2021.pdf"
pdf_data=tabula.read_pdf(pdf_path, pages=61)[0]

# Data Engineering

## Extract Column Name from the first two columns

In [4]:
row_desc_ser=ex.extract_col_nm(pdf_data)
row_desc_ser.replace('', np.nan, inplace=True)
row_desc_ser=row_desc_ser.dropna()
row_desc_ser

Unnamed: 2     average_balance
2021                yield_rate
Unnamed: 4            interest
Unnamed: 6     average_balance
2020                yield_rate
Unnamed: 8            interest
Unnamed: 10    average_balance
2019                yield_rate
Unnamed: 13           interest
dtype: object

In [6]:
# Delete the first two rows
pdf_data=pdf_data.drop([0,1]).reset_index(drop=True)
# Make the position column to index
pdf_data=pdf_data.rename(columns={"Unnamed: 0":"position"})
pdf_data["position"]=pdf_data["position"].apply(lambda x: x.split("(")[0])
pdf_data["position"]=pdf_data["position"].apply(ex.replace_special_char)
pdf_data["position"]=pdf_data["position"].apply(ex.snake_case)
pdf_data=pdf_data.set_index(keys="position",drop=True)
pdf_data

,Unnamed: 1,Unnamed: 2,2021,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,2020,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,2019,Unnamed: 11,Unnamed: 12,Unnamed: 13
position,,,,,,,,,,,,,,,,
cash_and_cash_equivalents,$,"14,236",0.13,%,$ 18,$,"11,348",0.30,%,$ 35,$,"9,667",2.27,%,$,219
restricted_cash,NaN,695,0.03,%,NM,NaN,438,0.45,%,2,NaN,620,2.24,%,NaN,14
other_short_term_investments,NaN,176,0.12,%,NM,NaN,"2,677",0.14,%,4,NaN,754,2.66,%,NaN,20
investment_securities,NaN,"8,713",2.09,%,182,NaN,"11,431",2.21,%,252,NaN,"7,603",2.35,%,NaN,179
loan_receivables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
credit_card_loans,NaN,"69,365",12.57,%,"8,717",NaN,"71,447",12.58,%,"8,985",NaN,"72,740",13.32,%,NaN,"9,690"
private_student_loans,NaN,"10,057",7.38,%,742,NaN,"9,890",7.63,%,754,NaN,"9,559",8.54,%,NaN,817
personal_loans,NaN,"6,945",12.64,%,878,NaN,"7,406",12.93,%,958,NaN,"7,522",13.07,%,NaN,983
other,NaN,"2,054",5.57,%,114,NaN,"1,660",6.35,%,105,NaN,"1,065",6.63,%,NaN,71


## Find Unit Columns

In [7]:
ser_unit_cols=ex.find_unit_cols(pdf_data)
ser_unit_cols

Unnamed: 1     [$]
Unnamed: 3     [%]
Unnamed: 5     [$]
Unnamed: 7     [%]
Unnamed: 9     [$]
Unnamed: 11    [%]
Unnamed: 12    [$]
dtype: object

## Extract Unit from a column 

In [8]:
temp_df=pdf_data

In [9]:
# Remove unit columns
temp_df=pdf_data
li_col=list(temp_df.columns)
[li_col.remove(col) for col in ser_unit_cols.keys()]
print(li_col)

['Unnamed: 2', '2021', 'Unnamed: 4', 'Unnamed: 6', '2020', 'Unnamed: 8', 'Unnamed: 10', '2019', 'Unnamed: 13']


In [10]:
li_new_col=[f"new_col_{str(li_col.index(col))}" for col in li_col]

In [11]:
for col in li_col:
    
    # get value and unit of the column
    col_val_unit=ex.get_val_unit(dataframe=temp_df,col_nm=col)

    # Add unit column whether we have unit in the unit column
    dict_check_unit=ex.check_unit(col_val_unit)
    
    if dict_check_unit["fg_unit"]:

        # get the index of the column
        idx_col=temp_df.columns.get_loc(col)

        if dict_check_unit["unit"]== "$":
            idx_col=idx_col
        elif dict_check_unit["unit"]== "%":
            idx_col=idx_col+1
        
        temp_df[col]=col_val_unit["val"]
        temp_df.insert(loc=idx_col, column=f'new_col: {col}', value=col_val_unit["unit"])

In [12]:
temp_df

,Unnamed: 1,Unnamed: 2,2021,Unnamed: 3,new_col: Unnamed: 4,Unnamed: 4,Unnamed: 5,Unnamed: 6,2020,Unnamed: 7,new_col: Unnamed: 8,Unnamed: 8,Unnamed: 9,Unnamed: 10,2019,Unnamed: 11,Unnamed: 12,Unnamed: 13
position,,,,,,,,,,,,,,,,,,
cash_and_cash_equivalents,$,"14,236",0.13,%,$,18,$,"11,348",0.30,%,$,35,$,"9,667",2.27,%,$,219
restricted_cash,NaN,695,0.03,%,NaN,NM,NaN,438,0.45,%,NaN,2,NaN,620,2.24,%,NaN,14
other_short_term_investments,NaN,176,0.12,%,NaN,NM,NaN,"2,677",0.14,%,NaN,4,NaN,754,2.66,%,NaN,20
investment_securities,NaN,"8,713",2.09,%,NaN,182,NaN,"11,431",2.21,%,NaN,252,NaN,"7,603",2.35,%,NaN,179
loan_receivables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
credit_card_loans,NaN,"69,365",12.57,%,NaN,"8,717",NaN,"71,447",12.58,%,NaN,"8,985",NaN,"72,740",13.32,%,NaN,"9,690"
private_student_loans,NaN,"10,057",7.38,%,NaN,742,NaN,"9,890",7.63,%,NaN,754,NaN,"9,559",8.54,%,NaN,817
personal_loans,NaN,"6,945",12.64,%,NaN,878,NaN,"7,406",12.93,%,NaN,958,NaN,"7,522",13.07,%,NaN,983
other,NaN,"2,054",5.57,%,NaN,114,NaN,"1,660",6.35,%,NaN,105,NaN,"1,065",6.63,%,NaN,71


In [14]:
ser_unit_cols=ex.find_unit_cols(temp_df)
ser_unit_cols

Unnamed: 1             [$]
Unnamed: 3             [%]
new_col: Unnamed: 4    [$]
Unnamed: 5             [$]
Unnamed: 7             [%]
new_col: Unnamed: 8    [$]
Unnamed: 9             [$]
Unnamed: 11            [%]
Unnamed: 12            [$]
dtype: object

In [15]:
dict_unit={}

for col in ser_unit_cols.keys():
    idx=list(temp_df.columns).index(col)
    if ser_unit_cols[col][0]== "$":
        dict_unit[temp_df.columns[idx+1]]="$"
    elif ser_unit_cols[col][0]== "%":
        dict_unit[temp_df.columns[idx-1]]="%"


temp_df=temp_df.drop(columns=list(ser_unit_cols.keys()))
temp_df

,Unnamed: 2,2021,Unnamed: 4,Unnamed: 6,2020,Unnamed: 8,Unnamed: 10,2019,Unnamed: 13
position,,,,,,,,,
cash_and_cash_equivalents,"14,236",0.13,18,"11,348",0.30,35,"9,667",2.27,219
restricted_cash,695,0.03,NM,438,0.45,2,620,2.24,14
other_short_term_investments,176,0.12,NM,"2,677",0.14,4,754,2.66,20
investment_securities,"8,713",2.09,182,"11,431",2.21,252,"7,603",2.35,179
loan_receivables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
credit_card_loans,"69,365",12.57,"8,717","71,447",12.58,"8,985","72,740",13.32,"9,690"
private_student_loans,"10,057",7.38,742,"9,890",7.63,754,"9,559",8.54,817
personal_loans,"6,945",12.64,878,"7,406",12.93,958,"7,522",13.07,983
other,"2,054",5.57,114,"1,660",6.35,105,"1,065",6.63,71


In [16]:
temp_df=temp_df.dropna(how="all")

In [17]:
li_dict_df=ex.separate_df(temp_df,3)

In [18]:
row_desc_ser

Unnamed: 2     average_balance
2021                yield_rate
Unnamed: 4            interest
Unnamed: 6     average_balance
2020                yield_rate
Unnamed: 8            interest
Unnamed: 10    average_balance
2019                yield_rate
Unnamed: 13           interest
dtype: object

In [19]:
li_dict_df=[
    dict(
        year=dict_df["year"],
        df=dict_df["df"].rename(
            columns=ex.assign_col_nms_from_desc(
                dict_df["df"],
                row_desc_ser
            )
        )
    )

    for dict_df in li_dict_df
]

In [21]:
li_dict_df[1]["df"]

,average_balance,yield_rate,interest
position,,,
cash_and_cash_equivalents,"11,348",0.30,35
restricted_cash,438,0.45,2
other_short_term_investments,"2,677",0.14,4
investment_securities,"11,431",2.21,252
credit_card_loans,"71,447",12.58,"8,985"
private_student_loans,"9,890",7.63,754
personal_loans,"7,406",12.93,958
other,"1,660",6.35,105
total_loan_receivables,"90,403",11.95,"10,802"
